In [15]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [6]:
spark = SparkSession.builder.appName('session-06').getOrCreate()
spark

In [7]:
train_df = spark.read.csv('../data/test1.csv',
                          header=True,
                          inferSchema=True)
train_df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
train_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
train_df.columns

['Name', 'age', 'Experience', 'Salary']

In [12]:
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'],
                                    outputCol='Independent Features')
output = feature_assembler.transform(train_df)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [14]:
final_data = output.select('Independent Features', 'Salary')
final_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [17]:
train_data, test_data = final_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

Exception ignored in: <function JavaWrapper.__del__ at 0x000002A3BC07EEE0>
Traceback (most recent call last):
  File "C:\Users\anupa\.android\trial\lib\site-packages\pyspark\ml\wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [18]:
regressor.coefficients

DenseVector([-258.8832, 1670.0508])

In [19]:
regressor.intercept

20543.147208121274

In [23]:
result_summary = regressor.evaluate(test_data)

In [25]:
result_summary.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16776.649746192812|
|          [24.0,3.0]| 20000|19340.101522842593|
+--------------------+------+------------------+



In [27]:
result_summary.meanAbsoluteError

1218.2741116751095

In [28]:
result_summary.meanSquaredError

1795975.1604008244